In [ ]:
import os
import sys
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Get current working directory instead of __file__
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from ModelArchitecture.ConvNeXtTinyModel import ConvNeXtTinyWheatModelWithConfidence
from ModelArchitecture.DenseNetModel import DenseNet121WheatModel
from ModelArchitecture.EfficientNetV2Model import EfficientNetV2SWheatCountWithConfidence
from ModelArchitecture.RepVGGA1Model import RepVGGA1WheatModelWithConfidence
from dataLoaderFunc import loadSplitData, createLoader


In [ ]:
# ✅ Custom Gaussian NLL Loss
def gaussian_nll_loss(pred_mean, pred_logvar, target):
    precision = torch.exp(-pred_logvar)
    return torch.mean(precision * (target - pred_mean)**2 + pred_logvar)

# ✅ Laplace NLL Loss (Robust + Confidence-Aware)
def laplace_nll_loss(pred_mean, pred_logvar, target):
    scale = torch.exp(pred_logvar)  # predicted Laplace scale
    loss = torch.abs(target - pred_mean) / scale + pred_logvar
    return torch.mean(loss)

# ✅ Training Function
def train_model(model, train_loader, val_loader, optimizer, scheduler, device, fileName, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for batch_idx, (rgb_batch, dsm_batch, label_batch) in enumerate(train_loader):
            rgb_batch, dsm_batch, label_batch = rgb_batch.to(device), dsm_batch.to(device), label_batch.to(device)

            optimizer.zero_grad()
            output = model(rgb_batch, dsm_batch)  # output: [B, 2]
            pred_mean = output[:, 0]
            pred_logvar = output[:, 1]
            loss = gaussian_nll_loss(pred_mean, pred_logvar, label_batch.squeeze())
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            if batch_idx % 20 == 0:
                print(f"Epoch {epoch+1}/{num_epochs} | Batch {batch_idx}/{len(train_loader)} | Loss: {loss.item():.4f}")

        train_loss /= len(train_loader)

        # ✅ Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for rgb_batch, dsm_batch, label_batch in val_loader:
                rgb_batch, dsm_batch, label_batch = rgb_batch.to(device), dsm_batch.to(device), label_batch.to(device)
                output = model(rgb_batch, dsm_batch)
                pred_mean = output[:, 0]
                pred_logvar = output[:, 1]
                loss = gaussian_nll_loss(pred_mean, pred_logvar, label_batch.squeeze())
                val_loss += loss.item()

        val_loss /= len(val_loader)
        scheduler.step(val_loss)

        print(f"✅ Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # ✅ Save model
        torch.save(model.state_dict(), f"{fileName}{epoch+1}.pth")


# ✅ Full Training Function
def train_model_laplace(model, train_loader, val_loader, optimizer, scheduler, device, fileName,  num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for rgb_batch, dsm_batch, label_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            rgb_batch = rgb_batch.to(device)
            dsm_batch = dsm_batch.to(device)
            label_batch = label_batch.to(device)

            optimizer.zero_grad()
            output = model(rgb_batch, dsm_batch)  # [B, 2]
            pred_mean = output[:, 0]
            pred_logvar = output[:, 1]

            loss = laplace_nll_loss(pred_mean, pred_logvar, label_batch.squeeze())
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        # ✅ Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for rgb_batch, dsm_batch, label_batch in val_loader:
                rgb_batch = rgb_batch.to(device)
                dsm_batch = dsm_batch.to(device)
                label_batch = label_batch.to(device)

                output = model(rgb_batch, dsm_batch)
                pred_mean = output[:, 0]
                pred_logvar = output[:, 1]

                loss = laplace_nll_loss(pred_mean, pred_logvar, label_batch.squeeze())
                val_loss += loss.item()

        val_loss /= len(val_loader)
        scheduler.step(val_loss)

        print(f"✅ Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # ✅ Save Model
        torch.save(model.state_dict(), f"{fileName}{epoch+1}.pth")

In [ ]:
train_df, val_df, test_df = loadSplitData("C:/Users/pacha/Desktop/masterProj/MasterProj/Model_Creation/TraitPredictionModel/SPAD/RGB_DSM_SPAD.csv")
train_loader, val_loader, test_loader = createLoader(train_df, val_df, test_df, traitName = "SPAD")

In [ ]:
# Initialize
if torch.backends.mps.is_available():
    device = "mps"  # ✅ Use Apple Metal (Mac M1/M2)
    torch.set_default_tensor_type(torch.FloatTensor)
elif torch.cuda.is_available():
    device = "cuda"  # ✅ Use NVIDIA CUDA (Windows RTX 4060)
else:
    device = "cpu"  # ✅ Default to CPU if no GPU is available
print(f"✅ Using device: {device}")

In [ ]:
EfficientNetV2Model = EfficientNetV2SWheatCountWithConfidence().to(device)
optimizer = optim.Adam(EfficientNetV2Model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
train_model_laplace(EfficientNetV2Model, train_loader, val_loader, optimizer, scheduler, device, "EfficientNetV2LaplaceModel",  num_epochs=10)

In [ ]:
ConvNeXtTinyModel = ConvNeXtTinyWheatModelWithConfidence().to(device)
optimizer = optim.Adam(ConvNeXtTinyModel.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
train_model_laplace(ConvNeXtTinyModel, train_loader, val_loader, optimizer, scheduler, device, "ConvNeXtTinyLaplaceModel", num_epochs=10)  

In [ ]:
RepVGGA1Model = RepVGGA1WheatModelWithConfidence().to(device)
optimizer = optim.Adam(RepVGGA1Model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
train_model_laplace(RepVGGA1Model, train_loader, val_loader, optimizer, scheduler, device, "RepVGGA1LaplaceModel", num_epochs=10)

In [ ]:
DenseNetModel = DenseNet121WheatModel().to(device)
optimizer = optim.Adam(DenseNetModel.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
train_model_laplace(DenseNetModel, train_loader, val_loader, optimizer, scheduler, device, "DenseNetLaplaceModel", num_epochs=10)